# This will load the data from BOX into a DSA instance As well as coordinate metadata

In [4]:
import girder_client
from dotenv import load_dotenv
import os
import concurrent.futures


In [9]:
## Connect to the girder_client 
load_dotenv()

gc = girder_client.GirderClient(apiUrl="https://candygram.neurology.emory.edu/api/v1")
gc.authenticate(apiKey=os.getenv("GIRDER_API_KEY"))

{'_id': '638e2ae7cd1d7fad17b9dd1f'}

In [ ]:
# This folder contains the individual Case Folders from Dr. Kose to be loaded into the DSA
base_path = "/Users/dagutman/Downloads/RegisteredCases/"

rootDataFolder = [
    d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))
]
nestedInputFolderData = {}

for rdf in rootDataFolder:
    cur_base_path = os.path.join(base_path, rdf)
    directories = [
        d
        for d in os.listdir(cur_base_path)
        if os.path.isdir(os.path.join(cur_base_path, d))
    ]
    nestedInputFolderData[rdf] = directories

## No trailing slash or this fails
# Get the ID of the target folder for the images

mISIC_baseFolderPath = "/mISIC/BoxFileSet"
# mISIC_folderId = gc.resourceLookup(f"/collection{mISIC_baseFolderPath}")
# ### Convert the resouce path into an actual UID so I can use it in subsequent query


def getFolderByDSAPath(
    folderName, baseFolderPath=mISIC_baseFolderPath, createFolder=True
):
    ### Will get a folderID given a resouce path, will add an option to not create it
    ## if the folder does not exist
    try:
        folderInfo = gc.resourceLookup(f"/collection{baseFolderPath}/{folderName}")
        return folderInfo
    except:
        ## Create the folder --- this currently assumes the rootFolder already exists.
        rootFolderInfo = gc.resourceLookup(f"/collection{baseFolderPath}")
        folderInfo = gc.createFolder(
            rootFolderInfo["_id"], folderName, reuseExisting=True
        )

        return folderInfo


## Check to see if the resource paths exists for the imageTypes


def loadRCMDicomDataSet(rootFolder, dicomID, directorySet):
    dicomRootFolderPath = os.path.join(rootFolder, dicomID)

    

    getFolderByDSAPath(dicomID, rootFolder)

    # imageTypes = ["Confocal Images", "VivaStack", "VivaBlock", "Macroscopic Images"]
    # for imgType in imageTypes:
    #     imgFolder = getFolderByDSAPath(imgType, dicomRootFolderPath)
    for d in directorySet:
        ## The folderType is everything before the # as I want to group things
        imgFolderType = d.split("#")[0]
        imgTypeFolder = getFolderByDSAPath(imgFolderType, dicomRootFolderPath)

        ## Now create a subfolder for the actual image bundle, grouped by the imgFolderType
        imgFolder = gc.createFolder(imgTypeFolder["_id"], d, reuseExisting=True)

        ## Get a list of items already uploaded to the DSA to prevent reuploading files..
        itemsInFolder = list(gc.listItem(imgFolder["_id"]))

        ## Turn the itemList into names so I can use it as a filter below.
        uploadedFileNames = [x["name"] for x in itemsInFolder]

        localImgFolderPath = os.path.join(base_path, dicomID, d)
        localFileList = os.listdir(localImgFolderPath)
        for lf in localFileList:
            if lf not in uploadedFileNames:
                uploadedImageFile = gc.uploadFileToFolder(
                    imgFolder["_id"], filepath=os.path.join(localImgFolderPath, lf)
                )
                print(uploadedImageFile["name"])


### Check and make sure every image has a largeImage tag..

for dicomFolder in nestedInputFolderData:
    print(dicomFolder)
    loadRCMDicomDataSet( mISIC_baseFolderPath, dicomFolder, nestedInputFolderData[dicomFolder])



Case13


IsADirectoryError: [Errno 21] Is a directory: '/Users/dagutman/Downloads/RegisteredCases/Case13/V46/VivaBlock #5'

In [ ]:

imageList = gc.get(f"resource/64e7c0d2fbfabbf55f1704a2/items?type=folder&limit=0")


### interesting.. if # is in the name it seems to be weird
import concurrent.futures
from tqdm import tqdm


def process_image(item):
    if "largeImage" not in item:
        try:
            gc.post(f'item/{item["_id"]}/tiles')
        except Exception as e:
            print(e)
            pass


# Create a ThreadPoolExecutor with the desired number of worker threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit the process_image function for each item in parallel
    futures = [executor.submit(process_image, item) for item in imageList]

    # Use tqdm to create a progress bar
    progress_bar = tqdm(total=len(futures), desc="Processing")

    # Iterate over the completed futures
    for future in concurrent.futures.as_completed(futures):
        # Update the progress bar
        progress_bar.update(1)

    # Close the progress bar
    progress_bar.close()
